In [18]:
pip install pycountry


In [46]:
fig = px.choropleth(
    df,
    locations='Country',
    locationmode='country names',
    color='Download Range',
    hover_name='Country',
    hover_data={'Downloads': True, 'Download Range': True},
    color_discrete_sequence=px.colors.sequential.Plasma,
    category_orders={'Download Range': range_order},
    title="App Downloads by Country (Custom Ranges)"
)

fig.update_layout(
    legend_title_text='Download Ranges',
    geo=dict(
        showframe=False,
        showcoastlines=True,
        showland=True,
        landcolor="lightgray",
        coastlinecolor="gray",
        showcountries=True,
        countrycolor="white",
        projection_type='natural earth'
    ),
    margin=dict(l=0, r=0, t=50, b=0)
)


In [36]:
pip install dash dash-bootstrap-components plotly

In [48]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html
import dash_bootstrap_components as dbc

# Sample Data
emp_headcount_years = pd.DataFrame({
    'Year': [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025],
    'Headcount': [47, 183, 437, 1218, 1332, 1879, 2432, 3145, 3821]
})

gender_data = pd.DataFrame({
    'Gender': ['Male', 'Female', 'Non-binary'],
    'Count': [254, 235, 11]
})

attrition_data = pd.DataFrame({
    'Department': ['Compliance', 'Customer Ops', 'Engineering', 'Financial Crime', 'Product'] * 2,
    'Left_Company': ['No'] * 5 + ['Yes'] * 5,
    'Count': [83, 93, 77, 91, 102, 9, 10, 9, 18, 8]
})

satisfaction_data = pd.DataFrame({
    'Department': ['Compliance', 'Customer Ops', 'Engineering', 'Financial Crime', 'Product'],
    'Performance_Rating': [4.2, 3.8, 4.1, 4.0, 4.3],
    'Left_Company': [10, 20, 15, 18, 12]
})

performance_data = pd.DataFrame({
    'Year': np.arange(2015, 2026),
    'Performance_Score': [60, 65, 67, 70, 74, 78, 82, 85, 89, 91, 94]
})

liability_data = pd.DataFrame({
    'Year': [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025],
    'Total_Liabilities': [0.18, 1.08, 83.64, 498.72, 1592.40, 3407.81, 4656.16, 6212.13, 12103.67, 17050.91]
})

# 1. Employee Headcount (Lollipop)
fig1 = px.scatter(emp_headcount_years, x="Year", y="Headcount", title="Employee Headcount Over Years")
for _, row in emp_headcount_years.iterrows():
    fig1.add_shape(type='line', x0=row['Year'], y0=0, x1=row['Year'], y1=row['Headcount'],
                   line=dict(color='#FF7F0E', width=3))
fig1.update_traces(marker=dict(size=12, color='#FF7F0E', line=dict(width=2, color='black')))
fig1.update_layout(paper_bgcolor='black', plot_bgcolor='black', font_color='white')

# 2. Gender Pie Chart
fig2 = px.pie(gender_data, values='Count', names='Gender', title='Gender Distribution',
              color_discrete_sequence=px.colors.qualitative.Pastel)
fig2.update_layout(paper_bgcolor='black', font_color='white')

# 3. Attrition by Department
fig3 = px.bar(attrition_data, x='Department', y='Count', color='Left_Company',
              title='Attrition by Department', barmode='group',
              color_discrete_map={'No': 'blue', 'Yes': 'red'})
fig3.update_layout(paper_bgcolor='black', plot_bgcolor='black', font_color='white')

# 4. Satisfaction Chart
fig4 = px.scatter(satisfaction_data, x='Performance_Rating', y='Left_Company',
                  color='Department', size='Performance_Rating', title='Satisfaction by Department')
fig4.update_layout(paper_bgcolor='black', plot_bgcolor='black', font_color='white')

# 5. Performance Forecast
fig5 = go.Figure()
fig5.add_trace(go.Scatter(x=performance_data['Year'], y=performance_data['Performance_Score'],
                          mode='lines+markers', name='Actual', line=dict(color='lime')))
trend = np.poly1d(np.polyfit(performance_data['Year'], performance_data['Performance_Score'], 1))
future_years = np.arange(2025, 2031)
fig5.add_trace(go.Scatter(x=future_years, y=trend(future_years),
                          mode='lines+markers', name='Forecast',
                          line=dict(dash='dash', color='orange')))
fig5.update_layout(title='Performance Forecast Prediction', xaxis_title='Year', yaxis_title='Score',
                   paper_bgcolor='black', plot_bgcolor='black', font_color='white')

# 6. Total Liabilities
fig6 = px.line(liability_data, x='Year', y='Total_Liabilities', markers=True,
               title='Total Liabilities Over Time', line_shape='spline',
               color_discrete_sequence=['#FF851B'])
fig6.update_traces(line=dict(width=5), marker=dict(size=8, color='white', line=dict(width=2, color='orange')))
fig6.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white')

# Build Dash Layout
app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])

app.layout = dbc.Container([
    html.H1("Monzo Dashboard", className="text-center my-4 text-light"),

    dbc.Row([
        dbc.Col(dcc.Graph(figure=fig1), width=12)
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(figure=fig2), width=6),
        dbc.Col(dcc.Graph(figure=fig3), width=6)
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(figure=fig4), width=6),
        dbc.Col(dcc.Graph(figure=fig5), width=6)
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(figure=fig6), width=12)
    ])
], fluid=True, style={'background-color': 'black'})

if __name__ == '__main__':
    app.run_server(debug=True,jupyter_mode="external", port=8098)


Dash app running on http://127.0.0.1:8098/
